In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import warnings

warnings.filterwarnings(action = 'ignore')

###  법정동 -> 행정동
- 현행 법정동  체계는 1910년 실시한 토지 조사 사업에 바탕을 둔 것
- 지역 여건 변화에 쉽게 적응하고, 행정 능률과 주민 편의를 도모하기 위해 설치된 최일선 지방 행정 기관의 관할 구역은 행정동
- 기존 법정동 데이터를 **행정동 데이터로 맵핑 하여 분석 진행**

In [115]:
def dong_change(file_name):
    sample_df = pd.read_excel(file_name)
    sample_df.drop('STD_DAY',axis=1,inplace=True)

    change_df = pd.read_excel('행정구역분류.xlsx',header=1)

    seoul_df = change_df[change_df['시도']=='서울특별시']
    seoul_df = seoul_df[seoul_df['행정구역명'].str[-1] =='동']
    seoul_df = seoul_df[['시군구','행정구역명','법정동','법정동코드','행정기관코드','법정동 관할구역\n분할여부']]
    seoul_df.columns = ['GU','DONG','법정동','DONG_CODE','행정기관코드','분할여부']
    non_div = seoul_df[seoul_df['분할여부'].isnull()][['GU','DONG','DONG_CODE','행정기관코드']]

    remove_du = non_div.drop_duplicates(['행정기관코드'])

    tmp_sample_df = sample_df.drop(['GU','DONG'],axis=1)
    con_df1 = pd.merge(tmp_sample_df,non_div,on="DONG_CODE")

    non_div_result = con_df1.groupby('행정기관코드').sum()
    non_div_result = non_div_result.drop(['DONG_CODE'],axis=1)

    non_div_result = pd.merge(non_div_result, remove_du,right_on='행정기관코드',left_index=True)
    non_div_result.drop('DONG_CODE',inplace=True,axis=1)

    non_div_result.rename(columns = {'행정기관코드':'DONG_CODE'},inplace=True)

    div_df = seoul_df[seoul_df['분할여부']=='분할연계'][['GU','DONG','DONG_CODE','행정기관코드']]

    div_dong_count = div_df.groupby('DONG_CODE').count()[['행정기관코드']]
    div_dong_count.columns = ['DONG_COUNT']

    con_df2 = pd.merge(sample_df, div_dong_count,left_on='DONG_CODE',right_index=True)
    con_df2.set_index('DONG_CODE',inplace=True)

    con_df2 = con_df2.iloc[:,3:]
    cal_df = con_df2.div(con_df2['DONG_COUNT'], axis=0) 
    div_result_df = pd.merge(div_df,cal_df,left_on='DONG_CODE',right_index=True)
    div_result_df = div_result_df.drop(['DONG_CODE','DONG_COUNT'],axis=1)
    div_result_df.rename(columns = {'행정기관코드':'DONG_CODE'},inplace=True)

    non_div_result = non_div_result[div_result_df.columns]
    result_df = pd.concat([non_div_result, div_result_df],axis=0)
    result_sum = result_df.groupby('DONG_CODE').sum()
    
    remove_result_du =result_df.drop_duplicates(['DONG_CODE'])[['GU','DONG','DONG_CODE']]
    result_df2 = pd.merge(result_sum, remove_result_du,right_on='DONG_CODE',left_index=True)

    return result_df2

### 전처리 - (1)
- 강남에 성형외과가 매우 몰려있음.
- 성형외과 데이터 일부 제거

In [116]:
def ps_remove_count():
    df = pd.read_excel('HOSPITAL_DM.xlsx',sheet_name='워크시트 익스포트')
    df = df[~df['HOSPITAL_NAME'].str.contains('성형')] # 성형외과 제거
    tmp_df = df[['HOSPITAL_CODE','HOSPITAL_NAME','HOSPITAL_TYPE','ADD_STR','DONG','DONG_CODE']]
    
    std = pd.read_excel('행정구역분류.xlsx',sheet_name='행정동 및 법정동코드와의 연계표')
    std.columns = std.loc[0]
    std = std[1:]
    std_seoul = std[std['시도'] == '서울특별시']
    std_df = std_seoul[['시군구','행정동\n(행정기관명)','법정동','행정기관코드','법정동코드','법정동 관할구역\n분할여부']][1:]
    std_df.columns = ['시군구','행정동','법정동','행정기관코드','법정동코드','분할여부']
    
    data = tmp_df['DONG_CODE'].value_counts()
    x = pd.DataFrame(data)
    x.rename(columns={'DONG_CODE':'HOSPITAL_NUM'},inplace=True)
    x['DONG_CODE'] = x.index
    x.index = range(len(x))

    return x

### 전처리 - (2)
- 버스데이터 누락이 많음
- 새로운 데이터로 변경

In [117]:
def add_bus_data(df):
    df.drop('BUS_NUM',axis=1,inplace=True)
    copy_df = df.copy()
    copy_df['GU_DONG'] = copy_df['GU'] + copy_df['DONG']
    
    bus_df = pd.read_csv('동별_버스정보_수정.csv')
    merge_df = pd.merge(copy_df, bus_df)
    merge_df.drop('GU_DONG',axis=1,inplace=True)
    
    return merge_df

In [118]:
df = pd.read_excel('FINAL_NUM_1029.xlsx') # 법정동 데이터
copy_df = df.copy()

ps_rm_cnt = ps_remove_count()

merge = pd.merge(copy_df,ps_rm_cnt, how='outer',on='DONG_CODE')
df = merge.fillna(0)
del(df['HOSPITAL_NUM_x'])
df.rename(columns={'HOSPITAL_NUM_y':'HOSPITAL_NUM'},inplace=True)

ve_df = pd.read_csv("VEGAN.csv",encoding="cp949")
ve_cnt_df = ve_df.groupby('DONG_CODE').count()[['VEGAN_CODE']]
ve_cnt_df.columns=['VEGAN_CNT']
ve_merge = pd.merge(df, ve_cnt_df, how="left",left_on="DONG_CODE",right_index=True)
ve_merge = ve_merge.fillna(0)

file_name = "법정동_성형외과제외_비건추가_data.xlsx"
ve_merge.to_excel(file_name, index=False)

In [119]:
change_df = dong_change(file_name)

In [120]:
change_bus_df = add_bus_data(change_df)


In [121]:
pd.set_option('display.max_columns',35)

In [122]:
ad_dong_df = change_bus_df[['GU', 'DONG','DONG_CODE','ACADEMY_NUM', 'KINDER_NUM', 'FIRE_NUM', 'ELE_SCH_NUM', 'MID_SCH_NUM',
       'HIGH_SCH_NUM', 'CCTV_NUM', 'POLICE_NUM', 'BIKE_NUM', 'CAR_SHR_NUM',
       'SUBWAY_NUM', 'SAFE_DLVR_NUM', 'DPTM_NUM', 'ANI_HSPT_NUM', 'PHARM_NUM',
       'LEISURE_NUM', 'KIDS_NUM', 'SPORT_NUM', 'GYM_NUM', 'GOLF_NUM',
       'STARBUCKS_NUM', 'MC_NUM', 'CON_NUM', 'NOISE_VIBRATION_NUM',
       'CHILD_MED_NUM', 'CAFE_NUM', 'PARK_NUM', 'HOSPITAL_NUM','BUS_CNT','VEGAN_CNT']]

In [123]:
# 마트 행정동 데이터 -> 열끝에 추가 
mart_df = pd.read_csv("대형마트_1101.csv",index_col=0)
mart_df.reset_index(inplace=True)

In [124]:
ad_dong_df['RETAIL_NUM'] = mart_df['RETAIL_NUM']

In [125]:
# 코리빙 행정동 데이터 -> 열끝에 추가
coliving_df = pd.read_csv("coliving_행정동.csv",index_col=0)
ad_dong_df['COLIVING_NUM'] = coliving_df['cnt']

In [126]:
# MZ세대 이웃 행정동 데이터 -> 열끝에 추가
mz_neigh_df = pd.read_csv("MZ세대인구수.csv",index_col=0)
ad_dong_df['GU_DONG'] = ad_dong_df['GU'] + ad_dong_df['DONG']
mz_neigh_df['GU_DONG'] = mz_neigh_df['GU'] + mz_neigh_df['DONG']
mz_neigh_df.drop(['GU','DONG','20~24세','25~29세','30~34세','35~39세'],axis=1,inplace=True)
ad_dong_df =  pd.merge(ad_dong_df, mz_neigh_df, on="GU_DONG")
ad_dong_df.drop(['GU_DONG'],axis=1,inplace=True)
ad_dong_df

,GU,DONG,DONG_CODE,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,POLICE_NUM,BIKE_NUM,CAR_SHR_NUM,SUBWAY_NUM,SAFE_DLVR_NUM,DPTM_NUM,ANI_HSPT_NUM,...,KIDS_NUM,SPORT_NUM,GYM_NUM,GOLF_NUM,STARBUCKS_NUM,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM,HOSPITAL_NUM,BUS_CNT,VEGAN_CNT,RETAIL_NUM,COLIVING_NUM,인구수
0,종로구,청운효자동,1111051500,18.000000,0.000000,1.0,2.000000,2.000000,3.0,56.666667,2.000000,5.666667,1.000000,0.333333,1.000000,0.0,2.000000,...,0.000000,0.0,0.0,0.000000,0.333333,0.0,4.000000,18816.000000,7.333333,39.666667,1.666667,11.666667,14.0,5.000000,0.000000,0.0,3210
1,종로구,사직동,1111053000,19.000000,10.000000,0.0,2.000000,0.000000,0.0,41.666667,3.000000,11.666667,5.000000,1.333333,0.000000,0.0,1.000000,...,0.000000,0.0,18.0,3.000000,7.333333,0.0,20.000000,22848.000000,10.333333,100.666667,2.666667,74.666667,16.0,8.000000,2.000000,1.0,2749
2,종로구,삼청동,1111054000,2.000000,10.000000,0.0,0.000000,1.000000,1.0,22.000000,1.000000,7.000000,1.000000,1.000000,0.000000,0.0,0.000000,...,0.000000,1.0,3.0,0.000000,1.000000,0.0,2.000000,14112.000000,1.000000,52.000000,1.000000,7.000000,11.0,6.000000,0.000000,0.0,684
3,종로구,부암동,1111055000,24.000000,0.000000,1.0,4.000000,1.000000,1.0,41.000000,1.000000,5.000000,5.000000,0.000000,0.000000,0.0,3.000000,...,0.000000,0.0,2.0,2.000000,0.000000,0.0,8.000000,6048.000000,5.000000,30.000000,1.000000,4.000000,9.0,1.000000,0.000000,0.0,2585
4,종로구,평창동,1111056000,28.000000,0.000000,0.0,0.000000,0.000000,1.0,21.000000,1.000000,7.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,5.0,5.000000,1.000000,0.0,6.000000,4032.000000,10.000000,36.000000,0.000000,14.000000,37.0,2.000000,0.000000,0.0,4597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,강동구,성내2동,1174065000,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,0.333333,6.000000,6.666667,0.333333,0.666667,0.0,2.666667,...,0.666667,0.0,10.0,4.666667,1.333333,0.0,38.666667,1442.333333,4.000000,47.666667,0.000000,59.333333,11.0,2.333333,3.333333,0.0,8130
422,강동구,성내3동,1174066000,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,0.333333,6.000000,6.666667,0.333333,0.666667,0.0,2.666667,...,0.666667,0.0,10.0,4.666667,1.333333,0.0,38.666667,1442.333333,4.000000,47.666667,0.000000,59.333333,9.0,2.333333,3.333333,0.0,7120
423,강동구,길동,1174068500,100.000000,20.000000,1.0,6.000000,1.000000,0.0,119.000000,0.000000,9.000000,10.000000,1.000000,1.000000,0.0,8.000000,...,1.000000,0.0,18.0,9.000000,2.000000,0.0,76.000000,4327.000000,10.000000,63.000000,1.000000,120.000000,28.0,1.000000,8.000000,0.0,14486
424,강동구,둔촌1동,1174069000,38.000000,0.000000,0.0,2.000000,1.500000,1.0,32.500000,1.000000,5.500000,2.000000,0.500000,0.000000,0.0,1.500000,...,0.000000,0.0,4.0,3.000000,0.000000,0.5,17.000000,2163.500000,0.000000,17.500000,1.000000,14.500000,4.0,0.500000,0.000000,0.0,33


In [127]:
ad_dong_df = ad_dong_df.rename(columns = {"인구수" : 'MZ_POP_CNT'})
ad_dong_df

,GU,DONG,DONG_CODE,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,POLICE_NUM,BIKE_NUM,CAR_SHR_NUM,SUBWAY_NUM,SAFE_DLVR_NUM,DPTM_NUM,ANI_HSPT_NUM,...,KIDS_NUM,SPORT_NUM,GYM_NUM,GOLF_NUM,STARBUCKS_NUM,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM,HOSPITAL_NUM,BUS_CNT,VEGAN_CNT,RETAIL_NUM,COLIVING_NUM,MZ_POP_CNT
0,종로구,청운효자동,1111051500,18.000000,0.000000,1.0,2.000000,2.000000,3.0,56.666667,2.000000,5.666667,1.000000,0.333333,1.000000,0.0,2.000000,...,0.000000,0.0,0.0,0.000000,0.333333,0.0,4.000000,18816.000000,7.333333,39.666667,1.666667,11.666667,14.0,5.000000,0.000000,0.0,3210
1,종로구,사직동,1111053000,19.000000,10.000000,0.0,2.000000,0.000000,0.0,41.666667,3.000000,11.666667,5.000000,1.333333,0.000000,0.0,1.000000,...,0.000000,0.0,18.0,3.000000,7.333333,0.0,20.000000,22848.000000,10.333333,100.666667,2.666667,74.666667,16.0,8.000000,2.000000,1.0,2749
2,종로구,삼청동,1111054000,2.000000,10.000000,0.0,0.000000,1.000000,1.0,22.000000,1.000000,7.000000,1.000000,1.000000,0.000000,0.0,0.000000,...,0.000000,1.0,3.0,0.000000,1.000000,0.0,2.000000,14112.000000,1.000000,52.000000,1.000000,7.000000,11.0,6.000000,0.000000,0.0,684
3,종로구,부암동,1111055000,24.000000,0.000000,1.0,4.000000,1.000000,1.0,41.000000,1.000000,5.000000,5.000000,0.000000,0.000000,0.0,3.000000,...,0.000000,0.0,2.0,2.000000,0.000000,0.0,8.000000,6048.000000,5.000000,30.000000,1.000000,4.000000,9.0,1.000000,0.000000,0.0,2585
4,종로구,평창동,1111056000,28.000000,0.000000,0.0,0.000000,0.000000,1.0,21.000000,1.000000,7.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,5.0,5.000000,1.000000,0.0,6.000000,4032.000000,10.000000,36.000000,0.000000,14.000000,37.0,2.000000,0.000000,0.0,4597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,강동구,성내2동,1174065000,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,0.333333,6.000000,6.666667,0.333333,0.666667,0.0,2.666667,...,0.666667,0.0,10.0,4.666667,1.333333,0.0,38.666667,1442.333333,4.000000,47.666667,0.000000,59.333333,11.0,2.333333,3.333333,0.0,8130
422,강동구,성내3동,1174066000,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,0.333333,6.000000,6.666667,0.333333,0.666667,0.0,2.666667,...,0.666667,0.0,10.0,4.666667,1.333333,0.0,38.666667,1442.333333,4.000000,47.666667,0.000000,59.333333,9.0,2.333333,3.333333,0.0,7120
423,강동구,길동,1174068500,100.000000,20.000000,1.0,6.000000,1.000000,0.0,119.000000,0.000000,9.000000,10.000000,1.000000,1.000000,0.0,8.000000,...,1.000000,0.0,18.0,9.000000,2.000000,0.0,76.000000,4327.000000,10.000000,63.000000,1.000000,120.000000,28.0,1.000000,8.000000,0.0,14486
424,강동구,둔촌1동,1174069000,38.000000,0.000000,0.0,2.000000,1.500000,1.0,32.500000,1.000000,5.500000,2.000000,0.500000,0.000000,0.0,1.500000,...,0.000000,0.0,4.0,3.000000,0.000000,0.5,17.000000,2163.500000,0.000000,17.500000,1.000000,14.500000,4.0,0.500000,0.000000,0.0,33


In [128]:
ad_dong_df.to_csv("행정동_컬럼추가_최종ver.csv")